In [1]:
{-# LANGUAGE OverloadedStrings #-}

:!if [ -d solarized/ ]; then rm -rf solarized; fi
:!git clone https://github.com/altercation/solarized
:!cd solarized
:!git show --format=raw -s

Cloning into 'solarized'...

commit e40cd4130e2a82f9b03ada1ca378b7701b1a9110
tree ecd0e58d6832566540a30dfd4878db518d5451d0
parent ab3c5646b41de1b6d95782371289db585ba8aa85
author Trevor Bramble <inbox@trevorbramble.com> 1372482098 -0700
committer Trevor Bramble <inbox@trevorbramble.com> 1372482214 -0700

    add tmux by @seebi!

In [2]:
:!mv .git/objects/pack/* .
:!cat *.pack | git unpack-objects
:!rm -rf pack-*

In [3]:
:!git cat-file -p HEAD

tree ecd0e58d6832566540a30dfd4878db518d5451d0
parent ab3c5646b41de1b6d95782371289db585ba8aa85
author Trevor Bramble <inbox@trevorbramble.com> 1372482098 -0700
committer Trevor Bramble <inbox@trevorbramble.com> 1372482214 -0700

add tmux by @seebi!

In [4]:
:!cat .git/HEAD

ref: refs/heads/master

In [5]:
:!cat .git/refs/heads/master

e40cd4130e2a82f9b03ada1ca378b7701b1a9110

In [6]:
:!git cat-file -p e40cd4130e2a82f9b03ada1ca378b7701b1a9110

tree ecd0e58d6832566540a30dfd4878db518d5451d0
parent ab3c5646b41de1b6d95782371289db585ba8aa85
author Trevor Bramble <inbox@trevorbramble.com> 1372482098 -0700
committer Trevor Bramble <inbox@trevorbramble.com> 1372482214 -0700

add tmux by @seebi!

In [7]:
:!ls .git/objects/e4/0cd4130e2a82f9b03ada1ca378b7701b1a9110
:!cat .git/objects/e4/0cd4130e2a82f9b03ada1ca378b7701b1a9110

.git/objects/e4/0cd4130e2a82f9b03ada1ca378b7701b1a9110

®SÛ×^ ¿ÃY÷Ö¶1ÓÓèf`«zCÒB)b)='¯©
Ù'&fBÇ×ñ¹w8uûñÞ¹ÉÁëvývÓÊ¾¬{{ê<ýBðì«÷nÒùtØÿ1ä?cUíz¹ÃñÛL¶ûÈO

In [8]:
:!zlib-flate -uncompress < .git/objects/e4/0cd4130e2a82f9b03ada1ca378b7701b1a9110

commit 248 tree ecd0e58d6832566540a30dfd4878db518d5451d0
parent ab3c5646b41de1b6d95782371289db585ba8aa85
author Trevor Bramble <inbox@trevorbramble.com> 1372482098 -0700
committer Trevor Bramble <inbox@trevorbramble.com> 1372482214 -0700

add tmux by @seebi!

In [9]:
import qualified Codec.Compression.Zlib as Z (compress, decompress)
import           Data.ByteString.Lazy        (fromStrict, toStrict)
import           Data.ByteString             (ByteString)
import qualified Data.ByteString        as B

compress, decompress :: ByteString -> ByteString
compress   = toStrict . Z.compress   . fromStrict
decompress = toStrict . Z.decompress . fromStrict

In [10]:
commit <- B.readFile ".git/objects/e4/0cd4130e2a82f9b03ada1ca378b7701b1a9110"
decompress commit

"commit 248\NULtree ecd0e58d6832566540a30dfd4878db518d5451d0\nparent ab3c5646b41de1b6d95782371289db585ba8aa85\nauthor Trevor Bramble <inbox@trevorbramble.com> 1372482098 -0700\ncommitter Trevor Bramble <inbox@trevorbramble.com> 1372482214 -0700\n\nadd tmux by @seebi!\n"

In [11]:
import           Data.Attoparsec.ByteString (Parser)
import qualified Data.Attoparsec.ByteString.Char8 as AC

parsed :: Parser a -> ByteString -> a
parsed parser = either error id . AC.parseOnly parser

In [12]:
parseHeader :: Parser (ByteString, Int)
parseHeader = do
    objectType <- AC.takeTill AC.isSpace
    AC.space
    len <- AC.decimal
    AC.char '\NUL'
    return (objectType, len)

commit <- decompress <$> B.readFile ".git/objects/e4/0cd4130e2a82f9b03ada1ca378b7701b1a9110"

parsed parseHeader commit

("commit",248)

In [13]:
type Ref = ByteString

parseHexRef :: Parser Ref
parseHexRef = AC.take 40

In [14]:
data Commit = Commit
    { commitTree      :: Ref
    , commitParents   :: [Ref]
    , commitAuthor    :: ByteString
    , commitCommitter :: ByteString
    , commitMessage   :: ByteString
    } deriving (Eq, Show)

parseCommit = do
    cTree      <-           AC.string "tree"      *> AC.space *> parseHexRef                   <* AC.endOfLine
    cParents   <- AC.many' (AC.string "parent"    *> AC.space *> parseHexRef                   <* AC.endOfLine)
    cAuthor    <-           AC.string "author"    *> AC.space *> AC.takeTill (AC.inClass "\n") <* AC.endOfLine
    cCommitter <-           AC.string "committer" *> AC.space *> AC.takeTill (AC.inClass "\n") <* AC.endOfLine
    AC.endOfLine
    cMessage   <- AC.takeByteString
    return $ Commit cTree cParents cAuthor cCommitter cMessage

parsed (parseHeader *> parseCommit) commit

Commit {commitTree = "ecd0e58d6832566540a30dfd4878db518d5451d0", commitParents = ["ab3c5646b41de1b6d95782371289db585ba8aa85"], commitAuthor = "Trevor Bramble <inbox@trevorbramble.com> 1372482098 -0700", commitCommitter = "Trevor Bramble <inbox@trevorbramble.com> 1372482214 -0700", commitMessage = "add tmux by @seebi!\n"}

In [15]:
import Data.Monoid ((<>), mappend, mconcat)
import Data.Byteable

instance Byteable Commit where
    toBytes (Commit cTree cParents cAuthor cCommitter cMessage) = mconcat
        [                       "tree "      <> cTree      <> "\n"
        , mconcat (map (\ref -> "parent "    <> ref        <> "\n") cParents)
        ,                       "author "    <> cAuthor    <> "\n"
        ,                       "committer " <> cCommitter <> "\n"
        ,                                                     "\n"
        ,                                       cMessage
        ]

parsedCommit = parsed (parseHeader *> parseCommit) commit
(parsed parseCommit . toBytes $ parsedCommit) == parsedCommit

True

In [16]:
import Data.ByteString.UTF8 (fromString, toString)

withHeader :: ByteString -> ByteString -> ByteString
withHeader oType content = mconcat [oType, " ", fromString . show $ B.length content, "\NUL", content]

withHeader "commit" (toBytes parsedCommit)

"commit 248\NULtree ecd0e58d6832566540a30dfd4878db518d5451d0\nparent ab3c5646b41de1b6d95782371289db585ba8aa85\nauthor Trevor Bramble <inbox@trevorbramble.com> 1372482098 -0700\ncommitter Trevor Bramble <inbox@trevorbramble.com> 1372482214 -0700\n\nadd tmux by @seebi!\n"

In [17]:
import Data.Digest.Pure.SHA

hash :: ByteString -> Ref
hash = fromString . showDigest . sha1 . fromStrict

hash (withHeader "commit" (toBytes parsedCommit))

"e40cd4130e2a82f9b03ada1ca378b7701b1a9110"

In [18]:
:!git cat-file -p ecd0e58d6832566540a30dfd4878db518d5451d0

100644 blob e69de29bb2d1d6434b8b29ae775ad8c2e48c5391	.gitmodules
100644 blob ec00a76061539cf774614788270214499696f871	CHANGELOG.mkd
100644 blob f95aaf80007d225f00d3109987ee42ef2c2e0c0a	DEVELOPERS.mkd
100644 blob ee08d7e44f15108ef5359550399dad55955b56ca	LICENSE
100644 blob d18ee9450251ea1b9a02ebd4d6fce022df9eb5e4	README.md
040000 tree 1981c76881c6a14e14d067a44247acd1bf6bbc3a	adobe-swatches-solarized
040000 tree 825c732bdd3a62aeb543ca89026a26a2ee0fba26	apple-colorpalette-solarized
040000 tree 7bab2828df5de23262a821cc48fe0ccf8bd2a9ae	emacs-colors-solarized
040000 tree f5fe8c3e20b2577223f617683a52eac31c5c9f30	files
040000 tree 5b60111510dbb3d8560cf58a36a20a99fc175658	gedit
040000 tree 60c9df3d6e1994b76d72c061a02639af3d925655	gimp-palette-solarized
040000 tree 979cf43752e4d698c7b5b47cff665142a274c133	img
040000 tree 3ff6d431303b66cc50e45b6fabd72302f210aebc	intellij-colors-solarized
040000 tree 8f387a531ad08f146c86e4b6007b898064ad4d7f	iterm2-colors-solarized
040000 tree 1e37592e62c85909be4c5

In [19]:
decompress <$> B.readFile ".git/objects/ec/d0e58d6832566540a30dfd4878db518d5451d0"

"tree 1282\NUL100644 .gitmodules\NUL\230\157\226\155\178\209\214CK\139)\174wZ\216\194\228\140S\145\&100644 CHANGELOG.mkd\NUL\236\NUL\167`aS\156\247taG\136'\STX\DC4I\150\150\248q100644 DEVELOPERS.mkd\NUL\249Z\175\128\NUL}\"_\NUL\211\DLE\153\135\238B\239,.\f\n100644 LICENSE\NUL\238\b\215\228O\NAK\DLE\142\245\&5\149P9\157\173U\149[V\202\&100644 README.md\NUL\209\142\233E\STXQ\234\ESC\154\STX\235\212\214\252\224\"\223\158\181\228\&40000 adobe-swatches-solarized\NUL\EM\129\199h\129\198\161N\DC4\208g\164BG\172\209\191k\188:40000 apple-colorpalette-solarized\NUL\130\\s+\221:b\174\181C\202\137\STXj&\162\238\SI\186&40000 emacs-colors-solarized\NUL{\171((\223]\226\&2b\168!\204H\254\f\207\139\210\169\174\&40000 files\NUL\245\254\140> \178Wr#\246\ETBh:R\234\195\FS\\\159\&040000 gedit\NUL[`\DC1\NAK\DLE\219\179\216V\f\245\138\&6\162\n\153\252\ETBVX40000 gimp-palette-solarized\NUL`\201\223=n\EM\148\183mr\192a\160&9\175=\146VU40000 img\NUL\151\156\244\&7R\228\214\152\199\181\180|\255fQB\162t\193\&3400

In [20]:
import Data.ByteString.Base16 (encode)

parseBinRef :: Parser Ref
parseBinRef = encode <$> AC.take 20

data Tree = Tree { treeEntries :: [TreeEntry] } deriving (Eq, Show)

data TreeEntry = TreeEntry
    { treeEntryPerms :: ByteString
    , treeEntryName  :: ByteString
    , treeEntryRef   :: Ref
    } deriving (Eq, Show)

parseTreeEntry :: Parser TreeEntry
parseTreeEntry = do
    perms <- fromString <$> AC.many1' AC.digit
    AC.space
    name  <- AC.takeWhile (/='\NUL')
    AC.char '\NUL'
    ref   <- parseBinRef
    return $ TreeEntry perms name ref

parseTree :: Parser Tree
parseTree = Tree <$> AC.many' parseTreeEntry

tree <- decompress <$> B.readFile ".git/objects/ec/d0e58d6832566540a30dfd4878db518d5451d0"

parsedTree = parsed (parseHeader *> parseTree) tree
parsedTree

Tree {treeEntries = [TreeEntry {treeEntryPerms = "100644", treeEntryName = ".gitmodules", treeEntryRef = "e69de29bb2d1d6434b8b29ae775ad8c2e48c5391"},TreeEntry {treeEntryPerms = "100644", treeEntryName = "CHANGELOG.mkd", treeEntryRef = "ec00a76061539cf774614788270214499696f871"},TreeEntry {treeEntryPerms = "100644", treeEntryName = "DEVELOPERS.mkd", treeEntryRef = "f95aaf80007d225f00d3109987ee42ef2c2e0c0a"},TreeEntry {treeEntryPerms = "100644", treeEntryName = "LICENSE", treeEntryRef = "ee08d7e44f15108ef5359550399dad55955b56ca"},TreeEntry {treeEntryPerms = "100644", treeEntryName = "README.md", treeEntryRef = "d18ee9450251ea1b9a02ebd4d6fce022df9eb5e4"},TreeEntry {treeEntryPerms = "40000", treeEntryName = "adobe-swatches-solarized", treeEntryRef = "1981c76881c6a14e14d067a44247acd1bf6bbc3a"},TreeEntry {treeEntryPerms = "40000", treeEntryName = "apple-colorpalette-solarized", treeEntryRef = "825c732bdd3a62aeb543ca89026a26a2ee0fba26"},TreeEntry {treeEntryPerms = "40000", treeEntryName = "em

In [21]:
import Data.ByteString.Base16 (decode)

instance Byteable TreeEntry where
    toBytes (TreeEntry perms name ref) = mconcat [perms, " ", name, "\NUL", fst $ decode ref]

instance Byteable Tree where
    toBytes (Tree entries) = mconcat (map toBytes entries)

(parsed parseTree . toBytes $ parsedTree) == parsedTree

True

In [22]:
:!git cat-file -p ec00a76061539cf774614788270214499696f871

Solarized Changelog

## Current release 1.0.0beta2

1.0.0beta2
----------

### Summary

Switch to the alternative red hue (final and only hue change), included a whole
heap of new ports and updates to the existing Vim colorscheme. The list of all 
currently included ports, highlighted items are new, updates noted:

#### Editors & IDEs

*   \[UPDATED\] **Vim**
*   \[NEW\] ***Emacs***
*   \[NEW\] ***IntelliJ IDEA***
*   \[NEW\] ***NetBeans***
*   \[NEW\] ***SeeStyle theme for Coda & SubEthaEdit***
*   \[NEW\] ***TextMate***
*   \[NEW\] ***Visual Studio***

#### Terminal Emulators

* \[UPDATED\] **iTerm2 colorschemes**
* \[UPDATED\] **OS X Terminal.app colors**
* \[UPDATED\] **Xresources colors**

#### Other Applications

* \[UPDATED\] **Mutt mail client colorschemes**

#### Palettes

* \[UPDATED\] **Adobe Photoshop Swatches**
* \[UPDATED\] **Apple Color Picker Palette**
* \[UPDATED\] **Gimp Palette**


### Critical Changes

These changes may require you to change your configuration.

*  

In [23]:
blob <- decompress <$> B.readFile ".git/objects/ec/00a76061539cf774614788270214499696f871"
blob

"blob 5549\NULSolarized Changelog\n===================\n\n## Current release 1.0.0beta2\n\n1.0.0beta2\n----------\n\n### Summary\n\nSwitch to the alternative red hue (final and only hue change), included a whole\nheap of new ports and updates to the existing Vim colorscheme. The list of all \ncurrently included ports, highlighted items are new, updates noted:\n\n#### Editors & IDEs\n\n*   \\[UPDATED\\] **Vim**\n*   \\[NEW\\] ***Emacs***\n*   \\[NEW\\] ***IntelliJ IDEA***\n*   \\[NEW\\] ***NetBeans***\n*   \\[NEW\\] ***SeeStyle theme for Coda & SubEthaEdit***\n*   \\[NEW\\] ***TextMate***\n*   \\[NEW\\] ***Visual Studio***\n\n#### Terminal Emulators\n\n* \\[UPDATED\\] **iTerm2 colorschemes**\n* \\[UPDATED\\] **OS X Terminal.app colors**\n* \\[UPDATED\\] **Xresources colors**\n\n#### Other Applications\n\n* \\[UPDATED\\] **Mutt mail client colorschemes**\n\n#### Palettes\n\n* \\[UPDATED\\] **Adobe Photoshop Swatches**\n* \\[UPDATED\\] **Apple Color Picker Palette**\n* \\[UPDATED\\] **Gim

In [24]:
data Blob = Blob { blobContent :: ByteString } deriving (Eq, Show)

parseBlob :: Parser Blob
parseBlob = Blob <$> AC.takeByteString

parsedBlob = parsed (parseHeader *> parseBlob) blob
parsedBlob

Blob {blobContent = "Solarized Changelog\n===================\n\n## Current release 1.0.0beta2\n\n1.0.0beta2\n----------\n\n### Summary\n\nSwitch to the alternative red hue (final and only hue change), included a whole\nheap of new ports and updates to the existing Vim colorscheme. The list of all \ncurrently included ports, highlighted items are new, updates noted:\n\n#### Editors & IDEs\n\n*   \\[UPDATED\\] **Vim**\n*   \\[NEW\\] ***Emacs***\n*   \\[NEW\\] ***IntelliJ IDEA***\n*   \\[NEW\\] ***NetBeans***\n*   \\[NEW\\] ***SeeStyle theme for Coda & SubEthaEdit***\n*   \\[NEW\\] ***TextMate***\n*   \\[NEW\\] ***Visual Studio***\n\n#### Terminal Emulators\n\n* \\[UPDATED\\] **iTerm2 colorschemes**\n* \\[UPDATED\\] **OS X Terminal.app colors**\n* \\[UPDATED\\] **Xresources colors**\n\n#### Other Applications\n\n* \\[UPDATED\\] **Mutt mail client colorschemes**\n\n#### Palettes\n\n* \\[UPDATED\\] **Adobe Photoshop Swatches**\n* \\[UPDATED\\] **Apple Color Picker Palette**\n* \\[UPDATED\\

In [25]:
instance Byteable Blob where
    toBytes (Blob content) = content

(parsed parseBlob . toBytes $ parsedBlob) == parsedBlob

True

In [26]:
:!git show-ref --tags

31ff7f5064824d2231648119feb6dfda1a3c89f5 refs/tags/v1.0.0beta1
a3037b428f29f0c032aeeeedb4758501bc32444d refs/tags/v1.0beta

In [27]:
:!git cat-file -p 31ff7f5064824d2231648119feb6dfda1a3c89f5

object 90581c7bfbcd279768580eec595d0ab3c094cc02
type commit
tag v1.0.0beta1
tagger Ethan Schoonover <es@ethanschoonover.com> 1300994142 -0700

Initial public beta release 1.0.0beta1

In [28]:
tag <- decompress <$> B.readFile ".git/objects/31/ff7f5064824d2231648119feb6dfda1a3c89f5"
tag

"tag 182\NULobject 90581c7bfbcd279768580eec595d0ab3c094cc02\ntype commit\ntag v1.0.0beta1\ntagger Ethan Schoonover <es@ethanschoonover.com> 1300994142 -0700\n\nInitial public beta release 1.0.0beta1\n"

In [29]:
data Tag = Tag
    { tagObject     :: Ref
    , tagType       :: ByteString
    , tagTag        :: ByteString
    , tagTagger     :: ByteString
    , tagAnnotation :: ByteString
    } deriving (Eq, Show)

parseTag :: Parser Tag
parseTag = do
    tObject     <- AC.string "object" *> AC.space *> parseHexRef                                                 <* AC.endOfLine
    tType       <- AC.string "type"   *> AC.space *> AC.choice (map AC.string ["commit", "tree", "blob", "tag"]) <* AC.endOfLine
    tTag        <- AC.string "tag"    *> AC.space *> AC.takeTill (AC.inClass "\n")                               <* AC.endOfLine
    tTagger     <- AC.string "tagger" *> AC.space *> AC.takeTill (AC.inClass "\n")                               <* AC.endOfLine
    AC.endOfLine
    tAnnotation <- AC.takeByteString
    return $ Tag tObject tType tTag tTagger tAnnotation

parsedTag = parsed (parseHeader *> parseTag) tag
parsedTag

Tag {tagObject = "90581c7bfbcd279768580eec595d0ab3c094cc02", tagType = "commit", tagTag = "v1.0.0beta1", tagTagger = "Ethan Schoonover <es@ethanschoonover.com> 1300994142 -0700", tagAnnotation = "Initial public beta release 1.0.0beta1\n"}

In [30]:
instance Byteable Tag where
    toBytes (Tag tObject tType tTag tTagger tAnnotation) = mconcat
        [ "object " <> tObject     <> "\n"
        , "type "   <> tType       <> "\n"
        , "tag "    <> tTag        <> "\n"
        , "tagger " <> tTagger     <> "\n"
        ,                             "\n"
        ,              tAnnotation
        ]

(parsed parseTag . toBytes $ parsedTag) == parsedTag

True

In [31]:
data GitObject
    = GitCommit Commit
    | GitTree   Tree
    | GitBlob   Blob
    | GitTag    Tag
    deriving (Eq, Show)

parseGitObject :: Parser GitObject
parseGitObject = do
    headerLen <- parseHeader
    case (fst headerLen) of
        "commit" -> GitCommit <$> parseCommit
        "tree"   -> GitTree   <$> parseTree
        "blob"   -> GitBlob   <$> parseBlob
        "tag"    -> GitTag    <$> parseTag
        _        -> error "not a git object"

instance Byteable GitObject where
    toBytes obj = case obj of
        GitCommit c -> withHeader "commit" (toBytes c)
        GitTree   t -> withHeader "tree"   (toBytes t)
        GitBlob   b -> withHeader "blob"   (toBytes b)
        GitTag    t -> withHeader "tag"    (toBytes t)

hashObject :: GitObject -> Ref
hashObject = hash . toBytes

In [32]:
hashObject . parsed parseGitObject <$> (decompress <$> B.readFile ".git/objects/31/ff7f5064824d2231648119feb6dfda1a3c89f5")

"31ff7f5064824d2231648119feb6dfda1a3c89f5"

In [33]:
import System.FilePath ((</>))

refPath :: FilePath -> Ref -> FilePath
refPath gitDir ref = let
   (dir,file) = splitAt 2 (toString ref)
   in gitDir </> "objects" </> dir </> file

refPath ".git" "31ff7f5064824d2231648119feb6dfda1a3c89f5"

".git/objects/31/ff7f5064824d2231648119feb6dfda1a3c89f5"

In [34]:
readObject :: FilePath -> Ref -> IO GitObject
readObject gitDir ref = do
    let path = refPath gitDir ref
    content <- decompress <$> B.readFile path
    return $ parsed parseGitObject content

readObject ".git" "31ff7f5064824d2231648119feb6dfda1a3c89f5"

GitTag (Tag {tagObject = "90581c7bfbcd279768580eec595d0ab3c094cc02", tagType = "commit", tagTag = "v1.0.0beta1", tagTagger = "Ethan Schoonover <es@ethanschoonover.com> 1300994142 -0700", tagAnnotation = "Initial public beta release 1.0.0beta1\n"})

In [35]:
import System.Directory (doesPathExist, createDirectoryIfMissing)
import System.FilePath  (takeDirectory)
import Control.Monad    (when, unless)

writeObject :: FilePath -> GitObject -> IO Ref
writeObject gitDir object = do
    let ref  =  hashObject object
    let path =  refPath gitDir ref
    exists   <- doesPathExist path
    unless exists $ do
        let dir = takeDirectory path
        createDirectoryIfMissing True dir
        B.writeFile path . compress $ toBytes object
    return ref

In [36]:
import Control.Monad    (forM)
import System.Directory (listDirectory)

allRefs <- do
    prefixes <- filter (\d -> length d == 2) <$> listDirectory ".git/objects/"
    concat <$> forM prefixes (\p ->
        map (fromString . (p++)) <$> listDirectory (".git/objects" </> p))

test <- forM allRefs $ \ref -> do
    obj  <- readObject  ".git" ref
    ref' <- writeObject ".git" obj
    return $ ref == ref'

and test

True